<a href="https://colab.research.google.com/github/dla9944/God_damn_deeplearning/blob/master/seq2seq__Eng2kor_%EC%98%81%EC%96%B4%EB%A5%BC_%ED%95%9C%EA%B8%80%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Tab-delimited Bilingual Sentence Pairs
# 출처 : http://www.manythings.org/anki

!wget https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt

--2023-01-20 00:35:34--  https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bigdata-young/ai_26th/main/data_dl/corpus.txt [following]
--2023-01-20 00:35:35--  https://raw.githubusercontent.com/bigdata-young/ai_26th/main/data_dl/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253511 (248K) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>] 247.57K  --.-KB/s    in 0.03s   

2023-01-20 00:35:35 (7.99 MB/s) - ‘corpus.txt’ saved [253511/253511]



In [ ]:
# text 파일 전처리

from string import punctuation
import string

In [ ]:

l = []

with open('./corpus.txt', 'r', encoding = 'utf-8') as f:
     lines = f.read().split('\n')
     for line in lines:

         txt = "".join(v for v in line if not v in string.punctuation).lower()
         l.append(txt)
         

In [ ]:
l[:5]

['go\t가', 'hi\t안녕', 'run\t뛰어', 'run\t뛰어', 'who\t누구']

In [ ]:
import numpy as np
import torch

from torch.utils.data.dataset import Dataset

In [ ]:
def get_BOW(corpus): 
    BOW = {'<SOS>' : 0, "<EOS>" : 1}

    for line in corpus:
        for word in line.split():
            if word not in BOW.keys():
               BOW[word] = len(BOW.keys())
    return BOW

In [ ]:
class Eng2Kor(Dataset):
   def __init__(self, path = './corpus.txt'):
       super().__init__()

       self.eng_corpus = [] 

       self.kor_corpus = [] 

       with open(path, 'r', encoding = 'utf-8') as f:
            lines = f.read().split('\n')

            for line in lines: 
                txt = "".join(v for v in line if not v in string.punctuation).lower()

                engtxt , kortxt = txt.split('\t')

                if len(engtxt.split()) <= 10 and len(kortxt.split()) <= 10:
                    self.eng_corpus.append(engtxt)
                    self.kor_corpus.append(kortxt)
       self.engBOW = get_BOW(self.eng_corpus)
       self.korBOW = get_BOW(self.kor_corpus)

   def gen_seq(self, line):
       seq = line.split()
       seq.append('<EOS>') 
       return seq
   
   def __len__(self):
       return len(self.eng_corpus)

   def __getitem__(self, i):
       data = np.array([
           self.engBOW[txt] for txt in self.gen_seq(self.eng_corpus[i])
       ])
       label = np.array([
           self.korBOW[txt] for txt in self.gen_seq(self.kor_corpus[i])
       ])
       
       return data, label

In [ ]:

def loader(dataset):
    for i in range(len(dataset)):
        data, label = dataset[i]

        yield torch.tensor(data), torch.tensor(label)


In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
   def __init__(self, input_size, hidden_size):
       super().__init__()

       self.embedding = nn.Embedding(input_size, hidden_size)

       self.gru = nn.GRU(hidden_size, hidden_size)    

   def forward(self, x, h):
       
       x = self.embedding(x).view(1, 1, -1)

       output, hidden = self.gru(x, h)
       return output, hidden

In [ ]:
class Decoder(nn.Module):
   def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length = 11): 
       super().__init__()

       self.embedding = nn.Embedding(output_size, hidden_size)

       self.attention = nn.Linear(hidden_size * 2, max_length)

       self.context = nn.Linear(hidden_size * 2, hidden_size)

       self.dropout = nn.Dropout(dropout_p)
       
       self.gru = nn.GRU(hidden_size, hidden_size)

       self.out = nn.Linear(hidden_size, output_size)

       self.relu = nn.ReLU()
       self.softmax = nn.LogSoftmax(dim = 1)

   def forward(self, x, h, encoder_outputs):
       x = self.embedding(x).view(1,1,-1)
       x = self.dropout(x)

       attn_weights = self.softmax(
           self.attention(torch.cat((x[0], h[0]), -1))
       )

       attn_applied = torch.bmm(
           attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0)
       )
       output = torch.cat((x[0], attn_applied[0]), 1)
       output = self.context(output).unsqueeze(0)
       output = self.relu(output)


       output, hidden = self.gru(output, h)

       output = self.out(output[0])

       return output

In [ ]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam
from torch.optim.adamw import AdamW


device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = Eng2Kor()

encoder = Encoder(input_size = len(dataset.engBOW), hidden_size = 64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p = 0.1).to(device)

encoder_optimizer = Adam(encoder.parameters(), lr = 0.001)
decoder_optimizer = Adam(encoder.parameters(), lr = 0.001)

In [ ]:
device

'cuda'

In [ ]:
# EPOCH 160일때 시간 소요는 얼마나 걸리는지 확인
EPOCHS = 160
for epoch in range(EPOCHS):
    iterator = tqdm(loader(dataset), total = len(dataset))
    total_loss = 0

    for data, label in iterator:
        data = torch.tensor(data, dtype = torch.long).to(device)
        label = torch.tensor(label, dtype = torch.long).to(device)

        encoder_hidden = torch.zeros(1, 1, 64).to(device)

        encoder_outputs = torch.zeros(11, 64).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        loss = 0

        for ei in range(len(data)):

            encoder_output, encoder_hidden = encoder(data[ei], encoder_hidden)

            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[0]]).to(device)

        decoder_hidden = encoder_hidden

        use_teacher_forcing = True if random.random() < 0.5 else False
        
        if use_teacher_forcing:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                
                target = torch.tensor(label[di], dtype = torch.long).to(device)
                target = torch.unsqueeze(target, dim = 0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)
                decoder_input = target
        else:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()


                target = torch.tensor(label[di], dtype = torch.long).to(device)
                target = torch.unsqueeze(target, dim = 0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)

                if decoder_input.item() == 1:
                   break

        total_loss += loss.item() / len(dataset)
        iterator.set_description(f'epoch : {epoch+1}. loss : {total_loss}')

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

torch.save(encoder.state_dict(), 'attn_enc.pt')
torch.save(decoder.state_dict(), 'attn_dec.pt')


  0%|          | 0/3592 [00:00<?, ?it/s]

<ipython-input-15-ca2b34dedcc3>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype = torch.long).to(device)
<ipython-input-15-ca2b34dedcc3>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype = torch.long).to(device)
<ipython-input-15-ca2b34dedcc3>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(label[di], dtype = torch.long).to(device)
<ipython-input-15-ca2b34dedcc3>:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

...

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

In [ ]:
!wget https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
!wget https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_dec.pt

--2023-01-20 03:06:59--  https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bigdata-young/ai_26th/main/etc/attn_enc.pt [following]
--2023-01-20 03:07:00--  https://raw.githubusercontent.com/bigdata-young/ai_26th/main/etc/attn_enc.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 727147 (710K) [application/octet-stream]
Saving to: ‘attn_enc.pt.1’

attn_enc.pt.1       100%[===================>] 710.10K  --.-KB/s    in 0.04s   

2023-01-20 03:07:00 (15.5 MB/s) - ‘attn_enc.pt.1’ saved [727147/727147]

--2023-01-20 03:07:00--  https://gi

In [ ]:

encoder.load_state_dict(torch.load("attn_enc.pt.1", map_location=device))
decoder.load_state_dict(torch.load("attn_dec.pt.1", map_location=device))

<All keys matched successfully>

In [ ]:

idx = random.randint(0, len(dataset))

input_sentence = dataset.eng_corpus[idx]
input_sentence

'well be right here'

In [ ]:

pred_sentence = ""

In [ ]:
data, label = dataset[idx]
data = torch.tensor(data, dtype = torch.long).to(device)
label = torch.tensor(label, dtype = torch.long).to(device) 

In [ ]:
data

tensor([165,  38, 251,  88,   1], device='cuda:0')

In [ ]:
label

tensor([ 213, 1293,  189, 1145,  443,    1], device='cuda:0')

In [ ]:

encoder_hidden = torch.zeros(1, 1, 64).to(device)
encoder_outputs = torch.zeros(11, 64).to(device)

In [ ]:
for ei in range(len(data)):

    encoder_output, encoder_hidden = encoder(data[ei], encoder_hidden)
    encoder_outputs[ei] = encoder_output[0, 0]

In [ ]:
encoder_outputs

tensor([[ 4.6860e-01, -5.5427e-02, -4.5561e-02, -8.3141e-02,  7.4216e-03,
          8.1421e-01,  8.4068e-02, -7.2774e-01,  7.7073e-01,  9.5079e-02,
         -7.4377e-01, -7.3433e-01, -7.2241e-01,  9.4146e-01,  3.2911e-02,
         -8.2010e-02,  6.8353e-01, -1.8503e-02, -8.6723e-01, -5.1345e-04,
         -1.3154e-01, -8.6209e-03, -7.9515e-01,  2.4373e-01, -1.4443e-01,
         -3.2074e-01,  2.2546e-01, -6.5786e-01, -5.7285e-02,  3.4455e-01,
         -3.4701e-01,  6.3201e-01,  5.3353e-01,  1.7554e-01, -4.9720e-01,
         -1.0441e-01, -2.1491e-01,  7.4364e-01,  1.9464e-01,  3.6034e-02,
          2.7860e-01,  4.6648e-03,  2.5971e-01, -6.2764e-01, -2.2865e-01,
          1.4088e-02, -1.6013e-01,  4.6153e-01, -3.1831e-01,  3.6844e-01,
         -3.6082e-02,  2.3460e-01,  6.7201e-03,  4.1095e-01, -1.9525e-01,
          4.9033e-02,  1.2512e-01, -6.3189e-01,  2.4941e-01, -3.5110e-01,
         -1.4408e-02, -7.4692e-01,  6.8292e-01,  2.0309e-01],
        [ 3.3086e-01, -3.3385e-01,  6.6391e-01, -1

In [ ]:

decoder_input = torch.tensor([[0]]).to(device)

decoder_hidden = encoder_hidden

In [ ]:
for di in range(11):
    
    decoder_output = decoder(decoder_input, decoder_hidden, encoder_outputs)

    topv, topi = decoder_output.topk(1)

    decoder_input = topi.squeeze().detach()

    if decoder_input.item() == 1:
       break
    
    pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "


In [ ]:
print(input_sentence)

well be right here


In [ ]:
print(pred_sentence)

우린 바로 여기 있을 거야 
